In [365]:
import pandas as pd

In [366]:
discounts = pd.read_csv('datasets/test/discounts_raw.csv')
discounts.sort_values(by = 'date_start', inplace = True)

In [367]:
discounts.dropna(inplace = True)

In [368]:
sales = pd.read_csv('datasets/test/sales_raw.csv')
sales.dropna(inplace = True)
sales = sales[sales['date']>'2023-11-30']
sales

,date,item_id,qnty
145170,2023-12-01,100545,3.0
145171,2023-12-01,100645,4.0
145172,2023-12-01,100683,1.0
145173,2023-12-01,100612,10.0
145174,2023-12-01,100165,15.0
...,...,...,...
147138,2023-12-31,100614,5.0
147139,2023-12-31,100140,4.0
147140,2023-12-31,100650,11.0
147141,2023-12-31,100182,1.0


In [369]:
merged = discounts.merge(sales, on = 'item_id', how = 'left')
merged.isna().sum()

item_id                     0
promo_typeCode              0
sale_price_before_promo     0
sale_price_time_promo       0
date_start                  0
date_end                    0
date                       49
qnty                       49
dtype: int64

In [370]:
merged['date_start'] = merged['date_start'].astype("datetime64[ns]")
merged['date_end'] = merged['date_end'].astype("datetime64[ns]")
merged['date'] = merged['date'].astype("datetime64[ns]")

In [371]:
merged.dropna(inplace = True)
merged.isna().sum()

item_id                    0
promo_typeCode             0
sale_price_before_promo    0
sale_price_time_promo      0
date_start                 0
date_end                   0
date                       0
qnty                       0
dtype: int64

In [372]:
merged['is_sale'] = 0
merged.loc[(merged['date']>merged['date_start']) & (merged['date']<merged['date_end']), 'is_sale'] = 1
merged.value_counts('is_sale')

is_sale
0    39811
1      320
dtype: int64

In [373]:
merged['week_day'] = merged['date'].dt.weekday
real_qnty = merged[(merged['date']>'2023-11-30') & (merged['item_id']==100045)]

In [374]:
merged['price'] = 0
merged.loc[merged['is_sale'] == 1, 'price'] = merged['sale_price_time_promo']
merged.loc[merged['is_sale'] == 0, 'price'] = merged['sale_price_before_promo']
merged = merged.loc[merged['item_id'] != 100530]
merged = merged.loc[merged['item_id'] != 100662]
merged = merged.loc[merged['item_id'] != 100663]
merged = merged.loc[merged['item_id'] != 100665]

In [375]:
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
import numpy as np

In [376]:
grouped_data_item = merged.groupby('item_id')
all_predictions = pd.DataFrame(columns=['date', 'item_id', 'predictions'])

In [377]:
for item_id, group_data in grouped_data_item:
    data = group_data.groupby('date')['qnty', 'week_day', 'is_sale'].sum()
    result = adfuller(data['qnty'])
    model = ARIMA(data['qnty'], order = (5,1,0)).fit()
    forecast = model.predict(start = 1, end = 31, exog=group_data[['week_day', 'is_sale']])
    real_qnty = data['qnty']
    
    #rmse = np.sqrt(mean_squared_error(real_qnty.iloc[0:31],forecast))
    #print(real_qnty)
    predictions = pd.DataFrame({
        'date': pd.date_range(start='1/1/2024', periods=31),
        'item_id': item_id,
        'predictions':forecast
    })
    print(predictions)
    all_predictions = pd.concat([all_predictions, predictions])

         date  item_id  predictions
1  2024-01-01   100045     3.999956
2  2024-01-02   100045     3.999935
3  2024-01-03   100045     8.569748
4  2024-01-04   100045     3.715252
5  2024-01-05   100045     7.171868
6  2024-01-06   100045     7.811048
7  2024-01-07   100045     2.210951
8  2024-01-08   100045     6.322641
9  2024-01-09   100045     7.015370
10 2024-01-10   100045    12.000187
11 2024-01-11   100045    10.430114
12 2024-01-12   100045    14.042809
13 2024-01-13   100045    14.704056
14 2024-01-14   100045    10.685524
15 2024-01-15   100045    12.615607
16 2024-01-16   100045    14.043804
17 2024-01-17   100045    12.570351
18 2024-01-18   100045    11.906504
19 2024-01-19   100045    13.402160
20 2024-01-20   100045    13.092963
21 2024-01-21   100045    12.303831
22 2024-01-22   100045    12.755722
23 2024-01-23   100045    13.171068
24 2024-01-24   100045    12.650023
25 2024-01-25   100045    12.607974
26 2024-01-26   100045    12.960247
27 2024-01-27   100045    12

         date  item_id  predictions
1  2024-01-01   100192    38.955879
2  2024-01-02   100192    70.148170
3  2024-01-03   100192    65.409878
4  2024-01-04   100192    33.498281
5  2024-01-05   100192    69.694212
6  2024-01-06   100192    76.046320
7  2024-01-07   100192    72.836925
8  2024-01-08   100192    57.210520
9  2024-01-09   100192    40.697846
10 2024-01-10   100192    83.801989
11 2024-01-11   100192   127.062319
12 2024-01-12   100192    27.140077
13 2024-01-13   100192    40.610268
14 2024-01-14   100192   107.800865
15 2024-01-15   100192   142.264523
16 2024-01-16   100192   140.164517
17 2024-01-17   100192    86.677832
18 2024-01-18   100192   127.999395
19 2024-01-19   100192   245.231394
20 2024-01-20   100192   210.137531
21 2024-01-21   100192   133.320242
22 2024-01-22   100192   124.497994
23 2024-01-23   100192   174.188009
24 2024-01-24   100192   227.377146
25 2024-01-25   100192   179.154154
26 2024-01-26   100192   136.296830
27 2024-01-27   100192   144

         date  item_id  predictions
1  2024-01-01   100354   114.481473
2  2024-01-02   100354   135.954952
3  2024-01-03   100354   147.910998
4  2024-01-04   100354   136.527183
5  2024-01-05   100354    83.651590
6  2024-01-06   100354    53.672157
7  2024-01-07   100354    38.559529
8  2024-01-08   100354    56.310121
9  2024-01-09   100354    74.644982
10 2024-01-10   100354   101.992981
11 2024-01-11   100354   122.115359
12 2024-01-12   100354   183.629145
13 2024-01-13   100354   191.866873
14 2024-01-14   100354   109.536696
15 2024-01-15   100354   110.853447
16 2024-01-16   100354   166.810768
17 2024-01-17   100354   190.176994
18 2024-01-18   100354   224.341888
19 2024-01-19   100354   462.797297
20 2024-01-20   100354   427.030888
21 2024-01-21   100354   351.417411
22 2024-01-22   100354   472.350538
23 2024-01-23   100354   386.864998
24 2024-01-24   100354   411.476820
25 2024-01-25   100354   438.207629
26 2024-01-26   100354   392.370465
27 2024-01-27   100354   428

         date  item_id  predictions
1  2024-01-01   100450    12.998251
2  2024-01-02   100450    12.998960
3  2024-01-03   100450    12.999704
4  2024-01-04   100450    13.000394
5  2024-01-05   100450    12.999579
6  2024-01-06   100450    26.301146
7  2024-01-07   100450    32.688218
8  2024-01-08   100450    36.592365
9  2024-01-09   100450    33.513675
10 2024-01-10   100450    28.134981
11 2024-01-11   100450    30.649518
12 2024-01-12   100450    32.751161
13 2024-01-13   100450    25.024961
14 2024-01-14   100450    14.222173
15 2024-01-15   100450    25.669563
16 2024-01-16   100450    11.582863
17 2024-01-17   100450    12.474479
18 2024-01-18   100450    19.828611
19 2024-01-19   100450    21.040645
20 2024-01-20   100450    20.588526
21 2024-01-21   100450    27.352307
22 2024-01-22   100450    18.796721
23 2024-01-23   100450    22.414674
24 2024-01-24   100450    23.220711
25 2024-01-25   100450    15.032552
26 2024-01-26   100450    24.872612
27 2024-01-27   100450    20

         date  item_id  predictions
1  2024-01-01   100473    62.774721
2  2024-01-02   100473    63.458214
3  2024-01-03   100473   190.548218
4  2024-01-04   100473   216.749052
5  2024-01-05   100473   251.501544
6  2024-01-06   100473   238.985126
7  2024-01-07   100473   233.311803
8  2024-01-08   100473   309.999923
9  2024-01-09   100473   278.776633
10 2024-01-10   100473   263.843065
11 2024-01-11   100473   217.712458
12 2024-01-12   100473   295.177664
13 2024-01-13   100473   252.660131
14 2024-01-14   100473   453.819457
15 2024-01-15   100473   382.844716
16 2024-01-16   100473   313.673730
17 2024-01-17   100473   290.831955
18 2024-01-18   100473   240.499954
19 2024-01-19   100473   262.657860
20 2024-01-20   100473   173.386332
21 2024-01-21   100473   160.440302
22 2024-01-22   100473   114.158442
23 2024-01-23   100473   135.344537
24 2024-01-24   100473    88.179220
25 2024-01-25   100473   113.804284
26 2024-01-26   100473   119.631461
27 2024-01-27   100473   106

         date  item_id  predictions
1  2024-01-01   100524   127.392028
2  2024-01-02   100524    96.613228
3  2024-01-03   100524    77.648727
4  2024-01-04   100524   132.418069
5  2024-01-05   100524   112.824322
6  2024-01-06   100524   102.124547
7  2024-01-07   100524   303.943924
8  2024-01-08   100524   518.354190
9  2024-01-09   100524    87.025387
10 2024-01-10   100524   222.860287
11 2024-01-11   100524   124.116482
12 2024-01-12   100524   161.902220
13 2024-01-13   100524   480.205415
14 2024-01-14   100524   501.062240
15 2024-01-15   100524     4.807369
16 2024-01-16   100524   145.443192
17 2024-01-17   100524    78.023359
18 2024-01-18   100524   216.991457
19 2024-01-19   100524   210.876809
20 2024-01-20   100524    71.193240
21 2024-01-21   100524    83.398465
22 2024-01-22   100524   130.826732
23 2024-01-23   100524   113.585871
24 2024-01-24   100524    61.333519
25 2024-01-25   100524    48.333343
26 2024-01-26   100524   174.354261
27 2024-01-27   100524   205

         date  item_id  predictions
1  2024-01-01   100553    81.524694
2  2024-01-02   100553   245.579106
3  2024-01-03   100553    90.191877
4  2024-01-04   100553   126.882338
5  2024-01-05   100553    52.960552
6  2024-01-06   100553    87.693696
7  2024-01-07   100553   213.993927
8  2024-01-08   100553    83.538920
9  2024-01-09   100553   131.461693
10 2024-01-10   100553   120.778919
11 2024-01-11   100553   105.735487
12 2024-01-12   100553   157.751554
13 2024-01-13   100553   261.447065
14 2024-01-14   100553   139.857440
15 2024-01-15   100553    53.422029
16 2024-01-16   100553    30.848560
17 2024-01-17   100553    65.021760
18 2024-01-18   100553   222.021620
19 2024-01-19   100553   164.688868
20 2024-01-20   100553   116.440687
21 2024-01-21   100553   165.192219
22 2024-01-22   100553    45.434510
23 2024-01-23   100553   110.526661
24 2024-01-24   100553   100.932033
25 2024-01-25   100553    90.300630
26 2024-01-26   100553   238.752737
27 2024-01-27   100553   659

         date  item_id  predictions
1  2024-01-01   100607    71.964634
2  2024-01-02   100607   111.070514
3  2024-01-03   100607   114.044939
4  2024-01-04   100607    94.321127
5  2024-01-05   100607    71.862511
6  2024-01-06   100607    55.596216
7  2024-01-07   100607    66.802917
8  2024-01-08   100607    64.569239
9  2024-01-09   100607    76.823171
10 2024-01-10   100607    46.865498
11 2024-01-11   100607    41.905920
12 2024-01-12   100607    16.235500
13 2024-01-13   100607    31.595728
14 2024-01-14   100607     4.151282
15 2024-01-15   100607    66.223290
16 2024-01-16   100607    74.481633
17 2024-01-17   100607    20.256532
18 2024-01-18   100607    29.628308
19 2024-01-19   100607    44.794632
20 2024-01-20   100607    41.797748
21 2024-01-21   100607   100.470864
22 2024-01-22   100607    51.404662
23 2024-01-23   100607    47.686372
24 2024-01-24   100607    29.708025
25 2024-01-25   100607    65.870715
26 2024-01-26   100607    47.333330
27 2024-01-27   100607    78

         date  item_id  predictions
1  2024-01-01   100645   163.892275
2  2024-01-02   100645    62.942346
3  2024-01-03   100645    95.773580
4  2024-01-04   100645   118.388172
5  2024-01-05   100645    53.289942
6  2024-01-06   100645    39.486088
7  2024-01-07   100645   164.926689
8  2024-01-08   100645    22.650909
9  2024-01-09   100645    55.477549
10 2024-01-10   100645    -6.437047
11 2024-01-11   100645   211.363159
12 2024-01-12   100645    13.292467
13 2024-01-13   100645    48.339660
14 2024-01-14   100645   117.787333
15 2024-01-15   100645   419.354036
16 2024-01-16   100645  1110.333089
17 2024-01-17   100645   879.502417
18 2024-01-18   100645   678.510241
19 2024-01-19   100645  1085.645437
20 2024-01-20   100645  1075.438748
21 2024-01-21   100645  1634.558134
22 2024-01-22   100645  1702.627904
23 2024-01-23   100645  3072.792978
24 2024-01-24   100645  4165.408706
25 2024-01-25   100645  2200.663277
26 2024-01-26   100645  2410.921754
27 2024-01-27   100645  2379

         date  item_id  predictions
1  2024-01-01   100668    34.000000
2  2024-01-02   100668    34.005233
3  2024-01-03   100668    34.000000
4  2024-01-04   100668    33.998757
5  2024-01-05   100668    34.000000
6  2024-01-06   100668    34.000000
7  2024-01-07   100668    34.000000
8  2024-01-08   100668    68.000664
9  2024-01-09   100668    95.262892
10 2024-01-10   100668   102.000392
11 2024-01-11   100668   120.561009
12 2024-01-12   100668   120.561699
13 2024-01-13   100668   126.742250
14 2024-01-14   100668   126.742960
15 2024-01-15   100668   126.948257
16 2024-01-16   100668   126.948761
17 2024-01-17   100668   125.531471
18 2024-01-18   100668   125.531715
19 2024-01-19   100668   124.342716
20 2024-01-20   100668   124.342773
21 2024-01-21   100668   123.752128
22 2024-01-22   100668   123.752097
23 2024-01-23   100668   123.582813
24 2024-01-24   100668   123.582765
25 2024-01-25   100668   123.598199
26 2024-01-26   100668   123.598165
27 2024-01-27   100668   123

In [378]:
real_qnty = pd.DataFrame(columns=['date', 'qnty'])
for item_id, group_data in grouped_data_item:
    data = group_data.groupby('date')['qnty', 'week_day', 'is_sale'].sum()  #100165
    real_data = pd.DataFrame({
            'qnty':data['qnty']})
    real_qnty = pd.concat([real_qnty, real_data])
real_data

,qnty
date,
2023-12-01,7.0
2023-12-02,14.0
2023-12-03,14.0
2023-12-04,28.0
2023-12-06,7.0
2023-12-07,7.0
2023-12-08,7.0
2023-12-09,14.0
2023-12-10,14.0


In [381]:
all_predictions.set_index(np.arange(0,2418))

,date,item_id,predictions
0,2024-01-01,100045,3.999956
1,2024-01-02,100045,3.999935
2,2024-01-03,100045,8.569748
3,2024-01-04,100045,3.715252
4,2024-01-05,100045,7.171868
...,...,...,...
2413,2024-01-27,100691,38.931803
2414,2024-01-28,100691,44.416460
2415,2024-01-29,100691,38.468031
2416,2024-01-30,100691,42.081013


In [384]:
test_predict = all_predictions.iloc[2387:]
test_predict = test_predict[(test_predict['date']!='2024-01-05')&(test_predict['date']!='2024-01-11')&(test_predict['date']!='2024-01-16')&(test_predict['date']!='2024-01-19')&(test_predict['date']!='2024-01-21')&(test_predict['date']!='2024-01-25')&(test_predict['date']!='2024-01-27')]
test_predict

,date,item_id,predictions
1,2024-01-01,100691,6.999621
2,2024-01-02,100691,11.362523
3,2024-01-03,100691,14.649062
4,2024-01-04,100691,21.331421
6,2024-01-06,100691,3.073026
7,2024-01-07,100691,14.498350
8,2024-01-08,100691,10.729713
9,2024-01-09,100691,13.509568
10,2024-01-10,100691,6.868978
12,2024-01-12,100691,14.264419


In [385]:
rmse = np.sqrt(mean_squared_error(real_data['qnty'],test_predict['predictions']))
rmse

15.959493646401947

In [387]:
all_predictions.to_csv('test_task.csv', sep = ';', index = False)